<a href="https://colab.research.google.com/github/Kahina-ainseur/EvoloPy-1/blob/master/Copie_de_Data_generateur_UNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import matplotlib.pyplot as plt

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pandas as pd
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.utils import Sequence

In [ ]:
########### METRICS

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [ ]:
######### MODEL U_net

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    """ Encoder """
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    """ Bridge """
    b1 = conv_block(p4, 1024)

    """ Decoder """
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    """ Outputs """
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    """ Model """
    model = Model(inputs, outputs)
    model.save('mon_modele.keras')

    return model

In [ ]:
# DataGenerator

In [ ]:
data_dir_train='/content/drive/MyDrive/donne/train'
for folder in os.listdir(data_dir_train):
  if folder == 'image':
    list_images = sorted(os.listdir(data_dir_train+ '/' + folder))
    print(list_images)
  else :
    list_masks = sorted(os.listdir(data_dir_train+ '/' + folder))


data_dir_val='/content/drive/MyDrive/donne/validation'
for folder in os.listdir(data_dir_val):
  if folder == 'image':
    list_images = sorted(os.listdir(data_dir_val+ '/' + folder))
    print(list_images)
  else :
    list_masks = sorted(os.listdir(data_dir_val+ '/' + folder))

data_dir_test='/content/drive/MyDrive/donne/test'
for folder in os.listdir(data_dir_test):
  if folder == 'image':
    list_images = sorted(os.listdir(data_dir_test+ '/' + folder))
    print(list_images)
  else :
    list_masks = sorted(os.listdir(data_dir_test+ '/' + folder))

['2_0003_org_x0_y0.png', '2_0003_org_x0_y0_180.png', '2_0003_org_x0_y0_270CW.png', '2_0003_org_x0_y0_90CW.png', '2_0003_org_x0_y192.png', '2_0003_org_x0_y192_180.png', '2_0003_org_x0_y192_270CW.png', '2_0003_org_x0_y192_90CW.png', '2_0003_org_x0_y384.png', '2_0003_org_x0_y384_180.png', '2_0003_org_x0_y384_270CW.png', '2_0003_org_x0_y384_90CW.png', '2_0003_org_x0_y576.png', '2_0003_org_x0_y576_180.png', '2_0003_org_x0_y576_270CW.png', '2_0003_org_x0_y576_90CW.png', '2_0003_org_x192_y0.png', '2_0003_org_x192_y0_180.png', '2_0003_org_x192_y0_270CW.png', '2_0003_org_x192_y0_90CW.png', '2_0003_org_x192_y192.png', '2_0003_org_x192_y192_180.png', '2_0003_org_x192_y192_270CW.png', '2_0003_org_x192_y192_90CW.png', '2_0003_org_x192_y384.png', '2_0003_org_x192_y384_180.png', '2_0003_org_x192_y384_270CW.png', '2_0003_org_x192_y384_90CW.png', '2_0003_org_x192_y576.png', '2_0003_org_x192_y576_180.png', '2_0003_org_x192_y576_270CW.png', '2_0003_org_x192_y576_90CW.png', '2_0003_org_x384_y0.png', '2_00

In [ ]:
class DataGenerator(Sequence):
    def __init__(self, data_dir, batch_size=8, image_size=(256, 256)):
        self.batch_size = batch_size
        self.image_size = image_size
        self.image_paths = sorted([os.path.join(data_dir, 'image', f) for f in os.listdir(os.path.join(data_dir, 'image'))])
        self.mask_paths = sorted([os.path.join(data_dir, 'mask', f) for f in os.listdir(os.path.join(data_dir, 'mask'))])

    def __len__(self):
        return len(self.image_paths) // self.batch_size

    def __getitem__(self, idx):
        batch_images = self.image_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_masks = self.mask_paths[idx * self.batch_size:(idx + 1) * self.batch_size]

        X = np.zeros((self.batch_size, *self.image_size, 1), dtype=np.float32)
        y = np.zeros((self.batch_size, *self.image_size, 1), dtype=np.float32)

        for i in range(len(batch_images)):
            image = cv2.imread(batch_images[i], cv2.IMREAD_GRAYSCALE)
            mask = cv2.imread(batch_masks[i], cv2.IMREAD_GRAYSCALE)

            image = cv2.resize(image, self.image_size)
            mask = cv2.resize(mask, self.image_size)

            X[i, ..., 0] = image / 255.0
            y[i, ..., 0] = mask / 255.0

        return X, y

# Utilisez le DataGenerator pour chaque set de données
train_gen = DataGenerator(data_dir_train, batch_size=8, image_size=(256, 256))
val_gen = DataGenerator(data_dir_val, batch_size=8, image_size=(256, 256))
test_gen = DataGenerator(data_dir_test, batch_size=8, image_size=(256, 256))

In [ ]:
# Définition du modèle U-Net
model = build_unet(input_shape=(256, 256, 1))

In [ ]:
# Compiler le modèle

from tensorflow.keras.optimizers import Adam

# Compilez le modèle
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['iou']
)


In [ ]:
# Entraîner le modèle
model.compile(
    optimizer='adam',
    loss=dice_loss,  # Utilisez directement la fonction si elle est définie
    metrics=[iou]    # Pareil pour les métriques
)

In [ ]:
# Entraîner le modèle
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10  # Nombre d'époques pour l'entraînement
)

model.save('mon_modele_entraine.h5')

Epoch 1/10
72/72 [==============================] - 2756s 38s/step - loss: 0.8242 - iou: 0.1036 - val_loss: 0.9811 - val_iou: 0.0096
Epoch 2/10
24/72 [=========>....................] - ETA: 30:01 - loss: 0.6217 - iou: 0.2596

In [ ]:
# Évaluer le modèle sur le jeu de test
results = model.evaluate(test_gen)
print("Test loss, Test accuracy:", results)

In [ ]:
# Faire des prédictions
predictions = model.predict(test_gen)

In [ ]:
##################################

In [ ]:
import matplotlib.pyplot as plt

def plot_sample(X, y, preds, binary_preds, ix=None):
    """Function to plot the results"""
    if ix is None:
        ix = np.random.randint(0, len(X))

    has_mask = y[ix].max() > 0

    fig, ax = plt.subplots(1, 4, figsize=(20, 10))
    ax[0].imshow(X[ix, ..., 0], cmap='gray')
    ax[0].set_title('Image')

    ax[1].imshow(y[ix].squeeze(), cmap='gray')
    ax[1].set_title('Mask')

    ax[2].imshow(preds[ix].squeeze(), vmin=0, vmax=1)
    ax[2].set_title('U-Net Prediction')

    ax[3].imshow(binary_preds[ix].squeeze(), vmin=0, vmax=1, cmap='gray')
    ax[3].set_title('Predicted binary mask')

    plt.show()


# Convertir les prédictions en masque binaire
binary_predictions = (predictions > 0.5).astype(np.uint8)

# Utiliser la fonction de tracé
plot_sample(np.array(test_gen.image_paths), np.array(test_gen.mask_paths), predictions, binary_predictions)


In [ ]:
##################################

In [ ]:
import matplotlib.pyplot as plt
import cv2
import os

def display_image_mask_pairs(image_dir, mask_dir, num_pairs=5):
    # Assurez-vous que les listes d'images et de masques sont triées et alignées
    image_files = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.png')])
    mask_files = sorted([os.path.join(mask_dir, f) for f in os.listdir(mask_dir) if f.endswith('.png')])

    # S'assurer que le nombre de paires n'excède pas le nombre d'images disponibles
    num_pairs = min(num_pairs, len(image_files))

    # Création des subplots
    fig, axs = plt.subplots(num_pairs, 2, figsize=(10, num_pairs * 5))

    for i in range(num_pairs):
        # Lire l'image et le masque
        img = cv2.imread(image_files[i])
        mask = cv2.imread(mask_files[i], cv2.IMREAD_GRAYSCALE)

        # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if num_pairs == 1:
            ax1, ax2 = axs[0], axs[1]
        else:
            ax1, ax2 = axs[i, 0], axs[i, 1]

        # Afficher l'image
        ax1.imshow(img)
        ax1.set_title('Image')
        ax1.axis('off')

        # Afficher le masque
        ax2.imshow(mask, cmap='gray')
        ax2.set_title('Mask')
        ax2.axis('off')

    plt.tight_layout()
    plt.show()

# Utilisation de la fonction
image_dir = '/content/drive/MyDrive/donne/train/image'
mask_dir = '/content/drive/MyDrive/donne/train/mask'
display_image_mask_pairs(image_dir, mask_dir, num_pairs=5)
